In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2 as cv
import random
import os

In [2]:
Image_Width=600
Image_Height=400
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
train_images = []
train_labels = []
train_path = './DermMel/train_sep/'

for root, dirs, files in os.walk(train_path):

    for name in dirs:
        direct = os.path.join(train_path, name)
        for filename in os.listdir(direct):
            img = cv.imread(os.path.join(train_path + "/" + name, filename))
            train_labels.append(name)
            img = cv.resize(img, Image_Size)
            train_images.append(img)

### Train Data

In [4]:
pd.DataFrame(columns=['Category', 'Number Of Items'], data=[['Non-Melanoma', train_labels.count('NotMelanoma')], ['Melanoma', train_labels.count('Melanoma')]])

,Category,Number Of Items
0,Non-Melanoma,5341
1,Melanoma,5341


In [5]:
test_images = []
test_labels = []
test_path = './DermMel/test/'

for root, dirs, files in os.walk(test_path):

    for name in dirs:
        direct = os.path.join(test_path, name)
        for filename in os.listdir(direct):
            img = cv.imread(os.path.join(test_path + "/" + name, filename))
            test_labels.append(name)
            img = cv.resize(img, Image_Size)
            test_images.append(img)

### Test Data

In [6]:
pd.DataFrame(columns=['Category', 'Number Of Items'], data=[['Non-Melanoma', train_labels.count('NotMelanoma')], ['Melanoma', train_labels.count('Melanoma')]])

,Category,Number Of Items
0,Non-Melanoma,5341
1,Melanoma,5341


In [7]:
validation_images = []
validation_labels = []
validation_path = './DermMel/train_sep/'

for root, dirs, files in os.walk(validation_path):

    for name in dirs:
        direct = os.path.join(validation_path, name)
        for filename in os.listdir(direct):
            img = cv.imread(os.path.join(validation_path + "/" + name, filename))
            validation_labels.append(name)
            img = cv.resize(img, Image_Size)
            validation_images.append(img)

### Validation Data

In [8]:
pd.DataFrame(columns=['Category', 'Number Of Items'], data=[['Non-Melanoma', train_labels.count('NotMelanoma')], ['Melanoma', train_labels.count('Melanoma')]])

,Category,Number Of Items
0,Non-Melanoma,5341
1,Melanoma,5341
